<a href="https://colab.research.google.com/github/dongjun-kor/Python/blob/main/karas_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=ae90a0daded005ffec802ad1732b8206d240586ad1605344b24b63a9b5b25bfc
  Stored in directory: /root/.cache/pip/wheels/a4/e8/45/842b3a39831261aef9154b907eacbc4ac99499a99ae829b06f
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=8e4675e6b4c2ff43834d11ee78b06e9d60194b383fac776d27738c0e0e1ce1b5
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=067029120ba9e79cc83a99b7a86c07eb835626aba912304c63049f711e142d8f
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=4473ef8ee428fd985de9317b1dae51016f594f9f1bd63ebb6690ff7c

In [2]:
import pandas as pd

In [3]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
path = 'gdrive/MyDrive/Colab Notebooks/'

In [5]:
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
import tensorflow as tf

import pandas as pd
import numpy as np
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

In [7]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

In [8]:
def copytree(src, dst, symlinks=False, ignore = None):
  for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
      shutil.copytree(s, d, symlinks, ignore)
    else:
      shutil.copy2(s, d)

In [9]:
#os.makedirs('bert')
# 존재하면 error 뜸

In [10]:
# wget을 활용해서 bert 모델 다운로드 가능
import os
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

import zipfile
import shutil
         
bert_zip = zipfile.ZipFile('multi_cased_L-12_H-768_A-12.zip')
bert_zip.extractall('bert')
 
bert_zip.close()

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree("bert/multi_cased_L-12_H-768_A-12", "bert")

--2022-05-15 09:24:51--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M   249MB/s    in 2.5s    

2022-05-15 09:24:53 (249 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]



In [11]:
#copytree(os.path.join(path, 'BERT_MODEL'), 'bert')

데이터 가져오기

In [12]:
#train = pd.read_csv("/content/gdrive/MyDrive/sIPCcode1.csv",encoding="euc-kr") # train dataset 가져오기
#test =  pd.read_csv("gdrive/MyDrive/Colab Notebooks/Dataset/news_test.csv") # test dataset 가져오기

In [13]:
train = pd.read_csv("/content/gdrive/MyDrive/dataset1.csv",encoding="euc-kr")
#test = pd.read_table("nsmc/"+"ratings_test.txt")

In [14]:
train

,ID,content,info,cat
0,A01B-001/00,Hand tools (edge trimmers for lawns A01G3/06 ),0,A
1,A01B-001/02,Spades; Shovels,0,A
2,A01B-001/04,with teeth,0,A
3,A01B-001/06,Hoes; Hand cultivators,0,A
4,A01B-001/08,with a single blade,0,A
...,...,...,...,...
67354,H01B-017/64,with conductive admixtures inserts or layers,7,H
67355,H01B-017/66,"Joining insulating bodies together, e.g. by b...",7,H
67356,H01B-019/00,Apparatus or processes specially adapted for m...,7,H
67357,H01B-019/02,Drying; Impregnating,7,H


In [95]:
SEQ_LEN = 128 # 문장 최대 길이
BATCH_SIZE = 32 # Batch size
EPOCHS = 3 # Epochs 
LR = 1e-5 # Learning Rate

pretrained_path = 'gdrive/MyDrive/Colab Notebooks/BERT_MODEL'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = 'content' # 분석할 문장의 Column
LABEL_COLUMN = 'info' # Label Column

In [96]:
LABEL_COLUMN = 'info' # Label Column

token_dict = {}

with codecs.open('/content/bert_base_uncased/vocab.txt', 'r', encoding='utf-8') as reader:
  for line in reader:
    token = line.strip()
    if "_" in token:
      token = token.replace("_","")
      token = "##" + token
    token_dict[token] = len(token_dict)

**허깅페이스에서 가져오기**

In [58]:
token_dict

{'[PAD]': 0,
 '[unused0]': 1,
 '[unused1]': 2,
 '[unused2]': 3,
 '[unused3]': 4,
 '[unused4]': 5,
 '[unused5]': 6,
 '[unused6]': 7,
 '[unused7]': 8,
 '[unused8]': 9,
 '[unused9]': 10,
 '[unused10]': 11,
 '[unused11]': 12,
 '[unused12]': 13,
 '[unused13]': 14,
 '[unused14]': 15,
 '[unused15]': 16,
 '[unused16]': 17,
 '[unused17]': 18,
 '[unused18]': 19,
 '[unused19]': 20,
 '[unused20]': 21,
 '[unused21]': 22,
 '[unused22]': 23,
 '[unused23]': 24,
 '[unused24]': 25,
 '[unused25]': 26,
 '[unused26]': 27,
 '[unused27]': 28,
 '[unused28]': 29,
 '[unused29]': 30,
 '[unused30]': 31,
 '[unused31]': 32,
 '[unused32]': 33,
 '[unused33]': 34,
 '[unused34]': 35,
 '[unused35]': 36,
 '[unused36]': 37,
 '[unused37]': 38,
 '[unused38]': 39,
 '[unused39]': 40,
 '[unused40]': 41,
 '[unused41]': 42,
 '[unused42]': 43,
 '[unused43]': 44,
 '[unused44]': 45,
 '[unused45]': 46,
 '[unused46]': 47,
 '[unused47]': 48,
 '[unused48]': 49,
 '[unused49]': 50,
 '[unused50]': 51,
 '[unused51]': 52,
 '[unused52]': 53,

In [98]:
class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
    if not self._cased:
      text = text

      text = text.lower()

    spaced = ''
    for ch in text:
      if self._is_punctuation(ch) or self._is_cjk_character(ch):
        spaced += ' ' + ch + ' '
      elif self._is_space(ch):
        spaced += ' '
      elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
        continue
      else:
        spaced += ch
      tokens = []
    for word in spaced.strip().split():
      tokens += self._word_piece_tokenize(word)
    return tokens

In [99]:
tokenizer = inherit_Tokenizer(token_dict)

In [115]:
tokenizer.tokenize('이번에 NH투자증권 공모전 나가면 좋은 경험이 될거야.')

Encoding(num_tokens=13, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [113]:
train['content'][16]

' for working in vineyards, orchards, or the like'

In [112]:
tokenizer.encode(train['content'][16])

Encoding(num_tokens=12, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [116]:
def convert_data(data_df):
  global tokenizer
  indices, targets = [], []
  for i in tqdm(range(len(data_df))):
    ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
    indices.append(ids)
    targets.append(data_df[LABEL_COLUMN][i])

  items = list(zip(indices, targets))

  indices, targets = zip(*items)
  indices = np.array(indices)
  return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)

  data_x, data_y = convert_data(data_df)

  return data_x, data_y

In [122]:
train_x, train_y = load_data(train)

  0%|          | 0/67359 [00:00<?, ?it/s]


TypeError: ignored

In [118]:
train_x

[array([[  101,  2192,  5906, ...,     0,     0,     0],
        [  101, 23288,  2015, ...,     0,     0,     0],
        [  101,  2007,  4091, ...,     0,     0,     0],
        ...,
        [  101, 14709,  2030, ...,     0,     0,     0],
        [  101, 17462,  1025, ...,     0,     0,     0],
        [  101, 12318,  1996, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [117]:
def sentence_convert_data(data):
  global tokenizer
  indices = []
  for i in tqdm(range(len(data))):
    print(tokenizer.tokenize(data[i]))
    ids, segments = tokenizer.encode(data[i], max_len = SEQ_LEN)
    indices.append(ids)

  items = indices
  indices = np.array(indices)
  return [indices, np.zeros_like(indices)]

def sentence_load_data(sentences): #sentence는 list input

  data_x = sentence_convert_data(sentences)

  return data_x

In [107]:
sentence_load_data(["오늘 날씨는 맑음", "내일 아침은 토스트를 먹을거야"])

100%|██████████| 2/2 [00:00<00:00, 3230.11it/s]

['[CLS]', '오', '##늘', '날', '##씨', '##는', '맑', '##음', '[SEP]']
['[CLS]', '내', '##일', '아', '##침', '##은', '토', '##스', '##트', '##를', '먹', '##을', '##거', '##야', '[SEP]']


[array([[101, 100, 100, 100, 100, 100, 100, 100, 102,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [101, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
         100, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0

In [121]:
# 모델정의
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training = True,
    trainable = True,
    seq_len = SEQ_LEN
)

KeyError: ignored

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=48)

In [ ]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output

  outputs = keras.layers.Dense(1, activation = 'relu', kernel_initializer = keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense) #밑에 두개의 layer를 제외하고 output layer를 붙여줌

  bert_model = keras.models.Model(inputs, outputs)
  Adam = tf.keras.optimizers.Adam(lr=0.00001)
  bert_model.compile(
      optimizer = Adam,
      loss = 'mae',
      metrics = ['accuracy'])
  
  return bert_model

In [ ]:
from IPython.display import SVG

In [ ]:
from tensorflow.keras.utils import model_to_dot

In [ ]:
SVG(model_to_dot(get_bert_finetuning_model(model), dpi = 65).create(prog='dot', format='svg'))

In [ ]:
bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs = 2, batch_size = 32, verbose = 1, shuffle=False)
bert_model.save_weights(path + "/bert1.h5")

In [ ]:
bert_model = get_bert_finetuning_model(model)
bert_model.load_weights(path + "/bert1.h5")

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x): #Pandas Dataframe을 인풋으로 받는다
    data_df = x
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)

    return data_x

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
def get_feature_map(model):
  inputs = model.input
  outputs = model.layers[-2].output
  feature_model = Model(inputs, outputs)
  return feature_model

In [ ]:
bert_feature = get_feature_map(bert_model)

In [ ]:
labels = train['info']

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x): #Pandas Dataframe을 인풋으로 받는다
    data_df = x
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x = predict_convert_data(data_df)

    return data_x

In [ ]:
train_set = predict_load_data(train)

In [ ]:
preds = bert_model.predict(train_set)

In [ ]:
bert_weight_list = bert_feature.predict(train_set)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [ ]:
bert_embedded = PCA(n_components=256).fit_transform(bert_weight_list)
bert_embedded = TSNE(n_components=2).fit_transform(bert_embedded)
bert_embedded

In [ ]:
g=pd.DataFrame(bert_embedded)
g.to_csv("bert_embedded.csv")